# Demonstration of pickled HBGR Model

This notebook shows a simple application of the HGBR patient pay predicting model that allows a doctor to compare the predicted patient pay for mulitple drugs that have been prescribed for a particular diagnosis.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle as pkl

In [5]:
# This cell is needed to create the diag_to_drugs dictionary below.

cmm_data = pd.read_csv('/home/ruggiec/Downloads/pharmacy_tx.csv')

cmm_data_no_dups = cmm_data.drop_duplicates()

cmm_data = cmm_data[cmm_data.rejected == 0].copy()

cmm_train, cmm_test = train_test_split(cmm_data, shuffle=True, 
                             random_state= 614, test_size=0.2)

The model is open and saved as pipe.

In [6]:
with open('hgbr_web', 'rb') as f:
    pipe = pkl.load(f)

In [7]:
# A dictionary of drugs prescribed for a given diagnosis.

diag_to_drug = {}
for diag in cmm_train.diagnosis.unique():
    diag_to_drug[diag] = cmm_train[cmm_train.diagnosis == diag]['drug'].unique()

In [8]:
def option_cost(diag, month, bin_num, pcn_num, group_num):
    features = np.array([[month, bin_num, pcn_num, group_num]], dtype='object')
    drugs = []
    pay = []
    for drug in diag_to_drug[diag]:
        drugs.append(drug)
        pay.append(pipe.predict(np.append(features, drug).reshape(-1,5))[0])
    df = {'drugs':drugs, 'predicted_pay':pay}
    return pd.DataFrame(df).sort_values('predicted_pay')

Below is an example for this simple application.

In [9]:
option_cost('B45.03', 5, 725700, '1UQC', np.nan)

,drugs,predicted_pay
2,generic todiadianic,49.365954
4,branded todiadianic,92.863260
0,branded pranic,221.182812
1,branded masonic,271.147075
3,branded monemodiase,541.926317
